M3. Research Fisher Vectors

In [5]:
import numpy as np
import pickle
import os

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import normalize

from utils.CodeTimer import CodeTimer
from utils.DatasetManager import DatasetManager
from descriptors.SIFT import DenseSIFT
from descriptors.VisualWords import VisualWords
from utils.Kernels import histogram_intersection_kernel
from fishervector import FisherVectorGMM

In [6]:
dataset = DatasetManager('../Databases/MIT_split')
train_img_paths, train_labels, test_img_paths, test_labels = dataset.load_dataset()
SAVE_PATH = '../SavePath/'

In [7]:
N_CLUSTERS = 896
STEP_SIZE = 16
DESC_SIZE = [8,16]
K_FOLDS = 5
PARAMETERS = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1],
    'gamma': ["scale"],
    'kernel': ["rbf", "sigmoid", histogram_intersection_kernel]}

In [ ]:
DenseSIFT = DenseSIFT()
train_desc_path = SAVE_PATH + 'desc_fisher' + os.sep + 'train.dat'
test_desc_path = SAVE_PATH  + 'desc_fisher' + os.sep + 'test.dat'
train_data_path = SAVE_PATH + 'data_fisher' + os.sep + 'train.dat'
test_data_path = SAVE_PATH  + 'data_fisher' + os.sep + 'test.dat'

train_desc = DenseSIFT.compute(train_img_paths, STEP_SIZE, DESC_SIZE)
test_desc = DenseSIFT.compute(test_img_paths, STEP_SIZE, DESC_SIZE)
# Save computed data
pickle.dump(train_desc, open(train_desc_path, 'wb'))
pickle.dump(test_desc, open(test_desc_path, 'wb'))

Example given in the github of fishervector library

In [9]:
import numpy as np
shape = [300, 20, 32] # e.g. SIFT image features
image_data = np.random.rand(300,256,64)
print(image_data.shape)
#image_data = np.concatenate([np.random.normal(np.ones(30), size=shape), np.random.normal(np.ones(30), size=shape)], axis=0)
from fishervector import FisherVectorGMM
fv_gmm_test = FisherVectorGMM(n_kernels=5).fit(image_data)
image_data_test = image_data[:10] # use a fraction of the data to compute the fisher vectors
fv = fv_gmm_test.predict(image_data_test)
print(fv.shape)

(300, 256, 64)
fitted GMM with 5 kernels
(10, 10, 64)


In [16]:
np.array(train_desc).shape

(1881, 512, 128)

In [9]:
STEP_SIZE = 16
DESC_SIZE = [8,16]
N_CLUSTERS = 256
N_KERNELS = 15

In [11]:
train_desc_path = SAVE_PATH + 'desc_fisher' + os.sep + 'train.dat'
test_desc_path = SAVE_PATH  + 'desc_fisher' + os.sep + 'test.dat'
train_data_path = SAVE_PATH + 'data_fisher' + os.sep + 'train.dat'
test_data_path = SAVE_PATH  + 'data_fisher' + os.sep + 'test.dat'
# Check for existing data files already computed
train_desc = pickle.load(open(train_desc_path, 'rb'))
test_desc = pickle.load(open(test_desc_path, 'rb'))
# Obtain Fisher Vectors for train and test sets
print("Compute FisherVectors Gaussian Mixture Model")
fv_gmm = FisherVectorGMM(n_kernels=N_KERNELS).fit(np.array(train_desc))

Compute FisherVectors Gaussian Mixture Model
fitted GMM with 15 kernels


In [17]:
train_data_500 = fv_gmm.predict(np.array(train_desc)[:500])
train_data_500_1000 = fv_gmm.predict(np.array(train_desc)[500:1000])
train_data_1000_1500 = fv_gmm.predict(np.array(train_desc)[1000:1500])
train_data_1500_end = fv_gmm.predict(np.array(train_desc)[1500:np.array(train_desc).shape[0]])
train_data = np.concatenate((train_data_500, train_data_500_1000, train_data_1000_1500, train_data_1500_end), axis=0)
pickle.dump(train_data, open(train_data_path, 'wb'))

test_data_500  = fv_gmm.predict(np.array(test_desc)[:500])
test_data_500_end  = fv_gmm.predict(np.array(test_desc)[500:np.array(test_desc).shape[0]])
test_data = np.concatenate((test_data_500, test_data_500_end), axis=0)
pickle.dump(test_data, open(test_data_path, 'wb'))

In [21]:
K_FOLDS = 5
PARAMETERS = {
    'C': [1e-1],
    'gamma': ["scale"],
    'kernel': ["rbf"]}

In [23]:
train_data = pickle.load(open(train_data_path, 'rb'))
test_data = pickle.load(open(test_data_path, 'rb'))

train_data = train_data.reshape(train_data.shape[0], 30*128)
test_data = test_data.reshape(test_data.shape[0], 30*128)

with CodeTimer("Train SVM"):
    print("Train SVM")
    cv = GridSearchCV(SVC(), param_grid=PARAMETERS, cv=K_FOLDS, n_jobs=-1, verbose=5)
    cv.fit(train_data, train_labels)

# Test SVM
with CodeTimer("Test SVM"):
    train_score = cv.score(train_data, train_labels)
    test_score = cv.score(test_data, test_labels)
        
print("Not Normalized Train accuracy score: {}\nTest accuracy score: {}\nBest params: {}\n".format(train_score, test_score, cv.best_params_))
print("All results: {}".format(cv.cv_results_))

Train SVM
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   24.0s remaining:   36.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   24.1s finished


Train SVM: 46.837796688079834 s
Test SVM: 19.98820209503174 s
Not Normalized Train accuracy score: 0.34502923976608185
Test accuracy score: 0.31226765799256506
Best params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}

All results: {'mean_fit_time': array([18.73633823]), 'std_fit_time': array([0.1077532]), 'mean_score_time': array([4.05638618]), 'std_score_time': array([0.07759704]), 'param_C': masked_array(data=[0.1],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=['scale'],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['rbf'],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}], 'split0_test_score': array([0.22691293]), 'split1_test_score': array([0.25329815]), 'split2_test_score': array([0.2393617]), 'split3_test_score': array([0.24266667]), 'split4_test_score': array([0.22